<a href="https://colab.research.google.com/github/Ajogeorge29/BONBID/blob/main/BONBID_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SimpleITK
!pip install numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 11.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
adc_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/1ADC_ss"
z_adc_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/2Z_ADC"
label_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/3LABEL"


In [ ]:


import SimpleITK as sitk
import numpy as np
import os
# Define a function to load .mha files
def load_mha_file(file_path):
    """
    Load an .mha file and return the image array along with the SimpleITK image object.
    """
    img = sitk.ReadImage(file_path)
    img_array = sitk.GetArrayFromImage(img)
    return img_array, img

adc_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/1ADC_ss"
z_adc_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/2Z_ADC"
label_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/3LABEL"

# Load an example .mha file for inspection
adc_example, adc_image = load_mha_file(os.path.join(adc_dir, "MGHNICU_010-VISIT_01-ADC_ss.mha"))
z_adc_example, z_adc_image = load_mha_file(os.path.join(z_adc_dir, "Zmap_MGHNICU_010-VISIT_01-ADC_smooth2mm_clipped10.mha"))
label_example, label_image = load_mha_file(os.path.join(label_dir, "MGHNICU_010-VISIT_01_lesion.mha"))

# Print basic info about the loaded data
print(f"ADC shape: {adc_example.shape}")
print(f"Z_ADC shape: {z_adc_example.shape}")
print(f"Label shape: {label_example.shape}")


ADC shape: (23, 128, 128)
Z_ADC shape: (23, 128, 128)
Label shape: (23, 128, 128)


/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/2Z_ADC/Zmap_MGHNICU_010-VISIT_01-ADC_smooth2mm_clipped10.mha

/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/3LABEL/MGHNICU_010-VISIT_01_lesion.mha

In [ ]:
# Normalization function
def normalize_image(img_array):
    mean = np.mean(img_array)
    std = np.std(img_array)
    return (img_array - mean) / std

# Apply normalization
adc_normalized = normalize_image(adc_example)
z_adc_normalized = normalize_image(z_adc_example)

# Resample function
def resample_image(img, new_spacing=[1.0, 1.0, 1.0]):
    original_spacing = img.GetSpacing()
    original_size = img.GetSize()

    new_size = [
        int(round(original_size[i] * (original_spacing[i] / new_spacing[i])))
        for i in range(3)
    ]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(new_spacing)
    resample.SetSize(new_size)
    resample.SetInterpolator(sitk.sitkLinear)
    resample.SetOutputDirection(img.GetDirection())
    resample.SetOutputOrigin(img.GetOrigin())
    resample.SetTransform(sitk.Transform())

    img_resampled = resample.Execute(img)
    return img_resampled

# Apply resampling to an ADC example
adc_resampled = resample_image(adc_image)

# Convert back to numpy array to inspect
adc_resampled_array = sitk.GetArrayFromImage(adc_resampled)
print(f"Resampled ADC shape: {adc_resampled_array.shape}")


Resampled ADC shape: (92, 200, 200)


In [ ]:
print("Files in ADC_ss:", os.listdir(adc_dir))
print("Files in Z_ADC:", os.listdir(z_adc_dir))
print("Files in LABEL:", os.listdir(label_dir))


Files in ADC_ss: ['MGHNICU_178-VISIT_01-ADC_ss.mha', 'MGHNICU_405-VISIT_01-ADC_ss.mha', 'MGHNICU_014-VISIT_01-ADC_ss.mha', 'MGHNICU_455-VISIT_01-ADC_ss.mha', 'MGHNICU_336-VISIT_01-ADC_ss.mha', 'MGHNICU_456-VISIT_01-ADC_ss.mha', 'MGHNICU_209-VISIT_01-ADC_ss.mha', 'MGHNICU_454-VISIT_01-ADC_ss.mha', 'MGHNICU_457-VISIT_01-ADC_ss.mha', 'MGHNICU_015-VISIT_01-ADC_ss.mha', 'MGHNICU_331-VISIT_01-ADC_ss.mha', 'MGHNICU_453-VISIT_01-ADC_ss.mha', 'MGHNICU_393-VISIT_01-ADC_ss.mha', 'MGHNICU_379-VISIT_01-ADC_ss.mha', 'MGHNICU_362-VISIT_01-ADC_ss.mha', 'MGHNICU_214-VISIT_01-ADC_ss.mha', 'MGHNICU_452-VISIT_01-ADC_ss.mha', 'MGHNICU_197-VISIT_01-ADC_ss.mha', 'MGHNICU_449-VISIT_01-ADC_ss.mha', 'MGHNICU_448-VISIT_01-ADC_ss.mha', 'MGHNICU_403-VISIT_01-ADC_ss.mha', 'MGHNICU_215-VISIT_01-ADC_ss.mha', 'MGHNICU_329-VISIT_01-ADC_ss.mha', 'MGHNICU_010-VISIT_01-ADC_ss.mha', 'MGHNICU_173-VISIT_01-ADC_ss.mha', 'MGHNICU_447-VISIT_01-ADC_ss.mha', 'MGHNICU_415-VISIT_01-ADC_ss.mha', 'MGHNICU_445-VISIT_01-ADC_ss.mha', 'M

In [ ]:
#Correction of missing or misplaced files

In [ ]:
# List files in each directory
adc_files = set(os.listdir(adc_dir))
z_adc_files = set(os.listdir(z_adc_dir))
label_files = set(os.listdir(label_dir))

print("Files in ADC_ss:", adc_files)
print("Files in Z_ADC:", z_adc_files)
print("Files in LABEL:", label_files)

Files in ADC_ss: {'MGHNICU_405-VISIT_01-ADC_ss.mha', 'MGHNICU_442-VISIT_01-ADC_ss.mha', 'MGHNICU_234-VISIT_01-ADC_ss.mha', 'MGHNICU_362-VISIT_01-ADC_ss.mha', 'MGHNICU_015-VISIT_01-ADC_ss.mha', 'MGHNICU_077-VISIT_01-ADC_ss.mha', 'MGHNICU_173-VISIT_01-ADC_ss.mha', 'MGHNICU_444-VISIT_01-ADC_ss.mha', 'MGHNICU_448-VISIT_01-ADC_ss.mha', 'MGHNICU_071-VISIT_01-ADC_ss.mha', 'MGHNICU_178-VISIT_01-ADC_ss.mha', 'MGHNICU_109-VISIT_01-ADC_ss.mha', 'MGHNICU_197-VISIT_01-ADC_ss.mha', 'MGHNICU_441-VISIT_01-ADC_ss.mha', 'MGHNICU_062-VISIT_01-ADC_ss.mha', 'MGHNICU_338-VISIT_01-ADC_ss.mha', 'MGHNICU_310-VISIT_01-ADC_ss.mha', 'MGHNICU_144-VISIT_01-ADC_ss.mha', 'MGHNICU_105-VISIT_01-ADC_ss.mha', 'MGHNICU_379-VISIT_01-ADC_ss.mha', 'MGHNICU_286-VISIT_01-ADC_ss.mha', 'MGHNICU_336-VISIT_01-ADC_ss.mha', 'MGHNICU_290-VISIT_01-ADC_ss.mha', 'MGHNICU_329-VISIT_01-ADC_ss.mha', 'MGHNICU_014-VISIT_01-ADC_ss.mha', 'MGHNICU_306-VISIT_01-ADC_ss.mha', 'MGHNICU_125-VISIT_01-ADC_ss.mha', 'MGHNICU_263-VISIT_01-ADC_ss.mha', 'M

In [ ]:
# Find mismatches between sets
missing_in_z_adc = adc_files - z_adc_files
missing_in_label = adc_files - label_files

print("Files in ADC but missing in Z_ADC:", missing_in_z_adc)
print("Files in ADC but missing in LABEL:", missing_in_label)

Files in ADC but missing in Z_ADC: {'MGHNICU_405-VISIT_01-ADC_ss.mha', 'MGHNICU_442-VISIT_01-ADC_ss.mha', 'MGHNICU_234-VISIT_01-ADC_ss.mha', 'MGHNICU_362-VISIT_01-ADC_ss.mha', 'MGHNICU_015-VISIT_01-ADC_ss.mha', 'MGHNICU_077-VISIT_01-ADC_ss.mha', 'MGHNICU_173-VISIT_01-ADC_ss.mha', 'MGHNICU_444-VISIT_01-ADC_ss.mha', 'MGHNICU_448-VISIT_01-ADC_ss.mha', 'MGHNICU_071-VISIT_01-ADC_ss.mha', 'MGHNICU_178-VISIT_01-ADC_ss.mha', 'MGHNICU_109-VISIT_01-ADC_ss.mha', 'MGHNICU_197-VISIT_01-ADC_ss.mha', 'MGHNICU_441-VISIT_01-ADC_ss.mha', 'MGHNICU_062-VISIT_01-ADC_ss.mha', 'MGHNICU_338-VISIT_01-ADC_ss.mha', 'MGHNICU_310-VISIT_01-ADC_ss.mha', 'MGHNICU_144-VISIT_01-ADC_ss.mha', 'MGHNICU_105-VISIT_01-ADC_ss.mha', 'MGHNICU_379-VISIT_01-ADC_ss.mha', 'MGHNICU_286-VISIT_01-ADC_ss.mha', 'MGHNICU_336-VISIT_01-ADC_ss.mha', 'MGHNICU_290-VISIT_01-ADC_ss.mha', 'MGHNICU_329-VISIT_01-ADC_ss.mha', 'MGHNICU_014-VISIT_01-ADC_ss.mha', 'MGHNICU_306-VISIT_01-ADC_ss.mha', 'MGHNICU_125-VISIT_01-ADC_ss.mha', 'MGHNICU_263-VISIT_

####Prefix Differences in Z_ADC:

The files in Z_ADC have the prefix "Zmap_" and a suffix "_smooth2mm_clipped10.mha".
To match these with files in ADC_ss, you’ll need to adjust for the prefix and suffix in your code.
Suffix Differences in LABEL:

The files in LABEL end with "_lesion.mha".
Similarly, the code needs to remove the suffix to match these files with those in ADC_ss.

In [ ]:
#Filematching corrections

In [ ]:
print("Base names in ADC_ss:", adc_files)
print("Base names in Z_ADC:", z_adc_files)
print("Base names in LABEL:", label_files)


Base names in ADC_ss: {'MGHNICU_405-VISIT_01-ADC_ss.mha', 'MGHNICU_442-VISIT_01-ADC_ss.mha', 'MGHNICU_234-VISIT_01-ADC_ss.mha', 'MGHNICU_362-VISIT_01-ADC_ss.mha', 'MGHNICU_015-VISIT_01-ADC_ss.mha', 'MGHNICU_077-VISIT_01-ADC_ss.mha', 'MGHNICU_173-VISIT_01-ADC_ss.mha', 'MGHNICU_444-VISIT_01-ADC_ss.mha', 'MGHNICU_448-VISIT_01-ADC_ss.mha', 'MGHNICU_071-VISIT_01-ADC_ss.mha', 'MGHNICU_178-VISIT_01-ADC_ss.mha', 'MGHNICU_109-VISIT_01-ADC_ss.mha', 'MGHNICU_197-VISIT_01-ADC_ss.mha', 'MGHNICU_441-VISIT_01-ADC_ss.mha', 'MGHNICU_062-VISIT_01-ADC_ss.mha', 'MGHNICU_338-VISIT_01-ADC_ss.mha', 'MGHNICU_310-VISIT_01-ADC_ss.mha', 'MGHNICU_144-VISIT_01-ADC_ss.mha', 'MGHNICU_105-VISIT_01-ADC_ss.mha', 'MGHNICU_379-VISIT_01-ADC_ss.mha', 'MGHNICU_286-VISIT_01-ADC_ss.mha', 'MGHNICU_336-VISIT_01-ADC_ss.mha', 'MGHNICU_290-VISIT_01-ADC_ss.mha', 'MGHNICU_329-VISIT_01-ADC_ss.mha', 'MGHNICU_014-VISIT_01-ADC_ss.mha', 'MGHNICU_306-VISIT_01-ADC_ss.mha', 'MGHNICU_125-VISIT_01-ADC_ss.mha', 'MGHNICU_263-VISIT_01-ADC_ss.mha

In [ ]:
import os

# Paths to directories in Google Drive
adc_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/1ADC_ss"
z_adc_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/2Z_ADC"
label_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/3LABEL"
output_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/Preprocessed"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Function to load .mha files (assuming this function is defined already in your environment)
def load_mha_file(file_path):
    import SimpleITK as sitk
    img = sitk.ReadImage(file_path)
    img_array = sitk.GetArrayFromImage(img)
    return img_array, img

# Function to resample (assuming this function is defined already in your environment)
def resample_image(img, new_spacing=[1.0, 1.0, 1.0]):
    import SimpleITK as sitk
    original_spacing = img.GetSpacing()
    original_size = img.GetSize()
    new_size = [int(round(original_size[i] * (original_spacing[i] / new_spacing[i]))) for i in range(3)]
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(new_spacing)
    resample.SetSize(new_size)
    resample.SetInterpolator(sitk.sitkLinear)
    resample.SetOutputDirection(img.GetDirection())
    resample.SetOutputOrigin(img.GetOrigin())
    resample.SetTransform(sitk.Transform())
    return resample.Execute(img)

# Adjust base names to match across directories
adc_files = {file_name.replace("-ADC_ss.mha", "") for file_name in os.listdir(adc_dir)}
z_adc_files = {file_name.replace("Zmap_", "").replace("-ADC_smooth2mm_clipped10.mha", "") for file_name in os.listdir(z_adc_dir)}
label_files = {file_name.replace("_lesion.mha", "") for file_name in os.listdir(label_dir)}

# Find common base names across all directories
common_files = adc_files.intersection(z_adc_files).intersection(label_files)
print("Common files across all directories:", common_files)

# Process each common file
for base_name in common_files:
    # Construct full paths using the base name
    adc_path = os.path.join(adc_dir, f"{base_name}-ADC_ss.mha")
    z_adc_path = os.path.join(z_adc_dir, f"Zmap_{base_name}-ADC_smooth2mm_clipped10.mha")
    label_path = os.path.join(label_dir, f"{base_name}_lesion.mha")

    # Load the images
    adc_image, adc_sitk = load_mha_file(adc_path)
    z_adc_image, z_adc_sitk = load_mha_file(z_adc_path)
    label_image, label_sitk = load_mha_file(label_path)

    # Resample images to target spacing
    adc_resampled = resample_image(adc_sitk)
    z_adc_resampled = resample_image(z_adc_sitk)
    label_resampled = resample_image(label_sitk)

    import SimpleITK as sitk

# Function to save processed .mha images
def save_mha_image(img_array, reference_image, output_path):
    """
    Save a numpy array as a .mha image file, using the reference image for metadata.
    """
    img_to_save = sitk.GetImageFromArray(img_array)
    img_to_save.CopyInformation(reference_image)  # Copy metadata from reference image
    sitk.WriteImage(img_to_save, output_path)


    # Save the processed files
    save_mha_image(sitk.GetArrayFromImage(adc_resampled), adc_sitk, os.path.join(output_dir, f"adc_resampled_{base_name}.mha"))
    save_mha_image(sitk.GetArrayFromImage(z_adc_resampled), z_adc_sitk, os.path.join(output_dir, f"z_adc_resampled_{base_name}.mha"))
    save_mha_image(sitk.GetArrayFromImage(label_resampled), label_sitk, os.path.join(output_dir, f"label_resampled_{base_name}.mha"))

    print(f"Processed and saved for {base_name}")


Common files across all directories: {'MGHNICU_231-VISIT_01', 'MGHNICU_437-VISIT_01', 'MGHNICU_435-VISIT_01', 'MGHNICU_415-VISIT_01', 'MGHNICU_077-VISIT_01', 'MGHNICU_215-VISIT_01', 'MGHNICU_444-VISIT_01', 'MGHNICU_109-VISIT_01', 'MGHNICU_454-VISIT_01', 'MGHNICU_348-VISIT_01', 'MGHNICU_379-VISIT_01', 'MGHNICU_144-VISIT_01', 'MGHNICU_438-VISIT_01', 'MGHNICU_312-VISIT_01', 'MGHNICU_197-VISIT_01', 'MGHNICU_014-VISIT_01', 'MGHNICU_173-VISIT_01', 'MGHNICU_072-VISIT_01', 'MGHNICU_105-VISIT_01', 'MGHNICU_178-VISIT_01', 'MGHNICU_452-VISIT_01', 'MGHNICU_125-VISIT_01', 'MGHNICU_275-VISIT_01', 'MGHNICU_010-VISIT_01', 'MGHNICU_441-VISIT_01', 'MGHNICU_209-VISIT_01', 'MGHNICU_272-VISIT_01', 'MGHNICU_449-VISIT_01', 'MGHNICU_323-VISIT_01', 'MGHNICU_062-VISIT_01', 'MGHNICU_331-VISIT_01', 'MGHNICU_338-VISIT_01', 'MGHNICU_376-VISIT_01', 'MGHNICU_305-VISIT_01', 'MGHNICU_214-VISIT_01', 'MGHNICU_101-VISIT_01', 'MGHNICU_064-VISIT_01', 'MGHNICU_303-VISIT_01', 'MGHNICU_456-VISIT_01', 'MGHNICU_071-VISIT_01', 'M

In [ ]:
# Preprocessing

In [ ]:
import os
import SimpleITK as sitk

# Paths to directories in Google Drive
adc_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/1ADC_ss"
z_adc_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/2Z_ADC"
label_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/3LABEL"
output_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/Preprocessed"

os.makedirs(output_dir, exist_ok=True)

def load_mha_file(file_path):
    img = sitk.ReadImage(file_path)
    img_array = sitk.GetArrayFromImage(img)
    return img_array, img

def resample_image(img, new_spacing=[1.0, 1.0, 1.0]):
    original_spacing = img.GetSpacing()
    original_size = img.GetSize()
    new_size = [int(round(original_size[i] * (original_spacing[i] / new_spacing[i]))) for i in range(3)]
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(new_spacing)
    resample.SetSize(new_size)
    resample.SetInterpolator(sitk.sitkLinear)
    resample.SetOutputDirection(img.GetDirection())
    resample.SetOutputOrigin(img.GetOrigin())
    resample.SetTransform(sitk.Transform())
    return resample.Execute(img)

# Updated save function
def save_mha_image(img, output_path):
    sitk.WriteImage(img, output_path)

# Process each common file
for base_name in common_files:
    adc_path = os.path.join(adc_dir, f"{base_name}-ADC_ss.mha")
    z_adc_path = os.path.join(z_adc_dir, f"Zmap_{base_name}-ADC_smooth2mm_clipped10.mha")
    label_path = os.path.join(label_dir, f"{base_name}_lesion.mha")

    adc_image, adc_sitk = load_mha_file(adc_path)
    z_adc_image, z_adc_sitk = load_mha_file(z_adc_path)
    label_image, label_sitk = load_mha_file(label_path)

    adc_resampled = resample_image(adc_sitk)
    z_adc_resampled = resample_image(z_adc_sitk)
    label_resampled = resample_image(label_sitk)

    save_mha_image(adc_resampled, os.path.join(output_dir, f"adc_resampled_{base_name}.mha"))
    save_mha_image(z_adc_resampled, os.path.join(output_dir, f"z_adc_resampled_{base_name}.mha"))
    save_mha_image(label_resampled, os.path.join(output_dir, f"label_resampled_{base_name}.mha"))

    print(f"Processed and saved for {base_name}")


Processed and saved for MGHNICU_231-VISIT_01
Processed and saved for MGHNICU_437-VISIT_01
Processed and saved for MGHNICU_435-VISIT_01
Processed and saved for MGHNICU_415-VISIT_01
Processed and saved for MGHNICU_077-VISIT_01
Processed and saved for MGHNICU_215-VISIT_01
Processed and saved for MGHNICU_444-VISIT_01
Processed and saved for MGHNICU_109-VISIT_01
Processed and saved for MGHNICU_454-VISIT_01
Processed and saved for MGHNICU_348-VISIT_01
Processed and saved for MGHNICU_379-VISIT_01
Processed and saved for MGHNICU_144-VISIT_01
Processed and saved for MGHNICU_438-VISIT_01
Processed and saved for MGHNICU_312-VISIT_01
Processed and saved for MGHNICU_197-VISIT_01
Processed and saved for MGHNICU_014-VISIT_01
Processed and saved for MGHNICU_173-VISIT_01
Processed and saved for MGHNICU_072-VISIT_01
Processed and saved for MGHNICU_105-VISIT_01
Processed and saved for MGHNICU_178-VISIT_01
Processed and saved for MGHNICU_452-VISIT_01
Processed and saved for MGHNICU_125-VISIT_01
Processed 